In [7]:
import torch
from models import ResNet18
from models import ResNeXt29_2x64d

import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

net = ResNeXt29_2x64d()
if device == 'cuda':
    net = net.to(device)
    net = torch.nn.DataParallel(net)
# Load weights from checkpoint.
checkpoint = torch.load("./checkpoint/ResNeXt29_2x64_89_44d.t7")
net.load_state_dict(checkpoint['state_dict'])


In [8]:
'''Train CIFAR10 with PyTorch.'''
from __future__ import print_function
import argparse
import os
from datetime import datetime
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision
import torchvision.transforms as transforms

import numpy as np
import torch.utils.data as data
from PIL import Image
import torch.nn.init as init


class CINIC10(data.Dataset):
    """`CINIC10 Dataset.

    Args:
        root (string): Root directory of dataset where directory
            ``cifhttp://140.110.18.35:31375/notebooks/tmp/final_1/Untitled1.ipynb?kernel_name=python3#ar-10-batches-py`` exists or will be saved to if download is set to True.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.

    """
 
    classes =['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    def __init__(self, root, train=True,transform=None, target_transform=None):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.train = train  # training set or test set
        self.img = []
        self.lab = []
        
        file_name = self.root
       # print(file_name)
        # now load the numpy arrays
        if os.path.exists(file_name):
            data= np.load(file_name)
            self.img = data['img']
            self.lab = data['label']
        else:
            print("It can't find .np")
        
            #self.test_data = self.test_data.transpose((0, 2, 3, 1))  # convert to HWC
            
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            tuple: (image, target) where target is index of the target class.
        """
       
        img, lab= self.img[index], self.lab[index]
      

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            lab = self.target_transform(lab)

        return img, lab


    def __len__(self):
        return len(self.img)


    def __repr__(self):
        fmt_str = 'Dataset ' + self.__class__.__name__ + '\n'
        fmt_str += '    Number of datapoints: {}\n'.format(self.__len__())
        tmp = 'train' if self.train is True else 'test'
        fmt_str += '    Split: {}\n'.format(tmp)
        fmt_str += '    Root Location: {}\n'.format(self.root)
        tmp = '    Transforms (if any): '
        fmt_str += '{0}{1}\n'.format(tmp, self.transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        tmp = '    Target Transforms (if any): '
        fmt_str += '{0}{1}'.format(tmp, self.target_transform.__repr__().replace('\n', '\n' + ' ' * len(tmp)))
        return fmt_str


In [9]:
test_dir = './test_img.npz'

cinic_mean = [0.47889522, 0.47227842, 0.43047404]
cinic_std = [0.24205776, 0.23828046, 0.25874835]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=cinic_mean, std=cinic_std)
])

testset = CINIC10(root=test_dir,transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=4)

classes = ('airplane', 'automobile', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')
criterion = nn.CrossEntropyLoss()


In [10]:
def test():
    ''' Final test of the best performing net on the testing dataset. '''
    #assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    #checkpoint = torch.load('./checkpoint/ckpt.t7')
    #net.load_state_dict(checkpoint['net'])
    net.eval()
    test_loss = 0
    correct = 0
   # total = 0
   # print('Test best performing net from epoch {} with accuracy {:.3f}%'.format(
   #     checkpoint['epoch'], checkpoint['acc']))
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            #total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
        print('Test: total accuracy is: %.2f %% , loss = %.3f \n' % ( 100 * correct / len(testset), test_loss/len(testset)) )


In [11]:
test()

Test: total accuracy is: 89.44 % , loss = 0.005 



In [35]:
print(checkpoint['acc'])
print(checkpoint['epoch'])

89.44
396
